In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('./data/transactions.csv', low_memory=False)

df.head()

,block_time,block_number,hash,value,tx_fee,nonce,from,to,method_id,sender,receiver,project,token_bought_symbol,token_sold_symbol,token_bought_amount,token_sold_amount,amount_usd,taker,tx_from,tx_to
0,2023-04-14 14:51:35.000 UTC,17046105.0,0x2afae7763487e60b893cb57803694810e6d3d136186a...,0,0.037013,0.0,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,NaN,0x60806040,0x0000000000000000000000000000000000000000,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-04-14 16:20:47.000 UTC,17046538.0,0xb38bba8bf8f61029a31e027c37fbc1065db784a39501...,0,0.001861,3.0,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0x6982508145454ce325ddbe47a25d4ec3d2311933,0xa9059cbb,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0x92fb5b4f8030103e0b11275c30965d1897ff23e5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-04-14 17:21:11.000 UTC,17046833.0,0x273894b35d8c30d32e1ffa22ee6aa320cc9f55f2adbb...,2000000000000000000,0.077539,4.0,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0xf305d719,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-14 17:34:47.000 UTC,17046900.0,0xc3760b499850b19d8f266dc10349d70f69c916982336...,1000000000000000,0.004451,5.0,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b,0x3593564c,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,uniswap-2,PEPE,WETH,1.951464e+11,0.001,2.079390,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0xfbfeaf0da0f2fde5c66df570133ae35f3eb58c9a,0xef1c6e67703c7bd7107eed8303fbe6ec2554bf6b
4,2023-04-14 19:46:59.000 UTC,17047545.0,0x08153c0546e4f73b178edae90d1e30fb519c7c49ff0d...,13000034453500519,0.009752,12056.0,0xaf2358e98683265cbd3a48509123d390ddf54534,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x7ff36ab5,0xa43fe16908251ee70ef74718545e4fe6c5ccec9f,0xaf2358e98683265cbd3a48509123d390ddf54534,uniswap-2,PEPE,WETH,2.519324e+12,0.013,27.131592,0xaf2358e98683265cbd3a48509123d390ddf54534,0xaf2358e98683265cbd3a48509123d390ddf54534,0x7a250d5630b4cf539739df2c5dacb4c659f2488d


In [14]:
# Swap rate
number_of_txs = len(df)
number_of_swap_txs = len(df[df['project'].notna()])

swap_rate = round((number_of_swap_txs / number_of_txs) * 100, 2)

swap_rate

88.52